# ECE 311 Lab 2: LSI Systems
## Due Date: 2/18 @ 11:59PM on Canvas
In this lab, we will explore Linear Shift-Invariant (LSI) systems and their properties with applications involving toy signals, image filtering, stock data, and even an example of a simple non-linear system. Let's get started!

In [ ]:
# import necessary libraries for this lab
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

from skimage.io import imread

%matplotlib inline

## Getting Started with Convolution

We remember from ECE 210, that convolution describes how any continuous-time input signal is processed by an LTI system. Given an input $x(t)$ and an LTI system's impulse response $h(t)$, the system output $y(t)$ is given by

$$
y(t) = x(t) * h(t).
$$

Recall that convolution for continous-time signals is defined as

$$
y(t) = \int_{\tau = -\infty}^{\infty}x(\tau)h(t-\tau)d\tau = \int_{\tau=-\infty}^{\infty}x(t-\tau)h(\tau)d\tau.
$$

You have learned in ECE 310 that discrete-time LTI systems also have an impulse response $h[n]$, which is the system response to a unit Kronecker delta $\delta[n]$ input. Thus, we can express the system output given an input signal via discrete-time convolution.

$$
y[n] = x[n] * h[n]
$$

$$
y[n] = \sum_{k=-\infty}^{\infty}x[k]h[n-k] = \sum_{k=-\infty}^{\infty}x[n-k]h[k]
$$

Like the width properties of continuous-time convolution, if $x$ is of length $N$ and $h$ is of length $M$, the result $y$ will be of length $N+M-1$. It is important to note that every LTI system can be represented by a convolution, every system that can be expressed as a convolution is fully described by its impulse response, and any system fully described by its impulse response must be LTI. This means the relationship between LTI systems, convolution, and impulse responses is an "if and only if" relationship; they all imply one another! This is something handy to keep in mind whenever you want to identify and describe an LTI system.

The key function we will use to perform convolutions is the $\textrm{convolve()}$ function in the $\textrm{scipy.signal}$ module. The usage of this function for an example system is as follows:

$$
x[n] = \delta[n]+2\delta[n-2]+3\delta[n-4]
$$

$$
y[n] = x[n]+3x[n-1]
$$

In [ ]:
x = np.array([1, 0, 2, 0, 3]) # input signal
h = np.array([1, 3]) # filter/system's impulse response
y = signal.convolve(x,h) # signal.convolve(in1,in2)

print(y) # verify this result by hand!

Note how we extracted the system's impulse response for the system's Linear Constant Coefficient Difference Equation (LCCDE): we simply replace each $x[n-k]$ with $\delta[n-k]$! This is equivalent to passing $\delta[n]$ as our input signal. Above, our first term takes the present input value and multiplies it by one, and the second term multiplies the most recent input by three. Intuitively, when we flip and shift our filter $h$ for the convolution, we will be applying this system to the input signal at each shift step. The $\textrm{signal.convolve()}$ function assumes the arrays that represent our signals begin at index zero.

# Exercise 1: Implementing LTI Systems

In the below code cell, implement the following LTI systems and plot the system response (using $\textrm{plt.stem()}$) to each of the listed input signals. Comment on the results in the following Markdown cell, i.e. compare and contrast how each system seems to affect each input signal. Remember to use the LCCDE for each system to infer its impulse response!

* System A: $y_a[n] = x[n]-x[n-1]$


* System B: $y_b[n] = \frac{1}{3}x[n]+\frac{1}{3}x[n-1]+\frac{1}{3}x[n-2]$



* $x_1[n] = u[n] - u[n-10], 0\leq n< 20$


* $x_2[n] = \sin\left(\frac{\pi}{20}n\right), 0\leq n< 40$

In [ ]:
# create input signals here:
# Hint: Use np.sin and np.pi!
x1 = np.array([1 for i in range(10)] + [0 for i in range(10)])
x2 = np.array([np.sin(np.pi/20 * i) for i in range (40)])
# System A
# plot result for x1
x1A = x1 - np.concatenate((np.array([0]), x1[:-1]), axis = 0)
n = np.array([i for i in range(x1A.size)])
plt.figure(figsize = (12,8))
plt.subplot(121)
plt.stem(n, x1A)
plt.title('x1 after system A')
plt.xlabel('n')
plt.ylabel('y[n]')
# plot result for x2
x2A = x2 - np.concatenate((np.array([0]), x2[:-1]), axis = 0)
n = np.array([i for i in range(x2A.size)])
plt.subplot(122)
plt.stem(n, x2A)
plt.title('x2 after system A')
plt.xlabel('n')
plt.ylabel('y[n]')

# System B
# plot result for x1
x1B = (1/3)*x1 + (1/3)*np.concatenate((np.array([0]), x1[:-1]), axis = 0) + (1/3)*np.concatenate((np.array([0,0]), x1[:-2]), axis = 0)
n = np.array([i for i in range(x1B.size)])
plt.figure(figsize = (12,8))
plt.subplot(121)
plt.stem(n, x1B)
plt.title('x1 after system B')
plt.xlabel('n')
plt.ylabel('y[n]')
# plot result for x2
x2B = (1/3)*x2 + (1/3)*np.concatenate((np.array([0]), x2[:-1]), axis = 0) + (1/3)*np.concatenate((np.array([0,0]), x2[:-2]), axis = 0)
n = np.array([i for i in range(x2B.size)])
plt.subplot(122)
plt.stem(n, x2B)
plt.title('x2 after system B')
plt.xlabel('n')
plt.ylabel('y[n]')

## Comments here:

(Consider how each system affects the flatter and faster-changing parts of the input signals. What do you think each system is doing?)
I think system A is differentiating the signal since by doing x[n] - x[n-1], we are calculating the change from one term to another. On the other hand, System B is averaging the closest 3 values, so I think System B smooths the input. 


# Exercise 2: Bitcoin Pricing Correction

One type of signal could be some non-physical 1D information. An example of this would be stock price or cryptocurrency price data. This data is notoriously noisy and can jump around unpredictably.

Remember that the systems we work with can be either causal or non-causal. A causal system only uses present and past information or values to calculate its present values, while a non-causal system can leverage future information. In this exercise, we will compare causal and non-causal versions of a system to smooth a day's worth of bitcoin price data. We have provided 24 hours of prices with pricing updates every minute (1440 points). The date in question is Christmas Eve Day (12/24), 2017.

In [ ]:
bitcoin_data = np.load('bitcoin-christmas.npy', allow_pickle=True)
n_points = len(bitcoin_data)
plt.figure(figsize = (8,6))
plt.plot(range(n_points), bitcoin_data)
plt.title('Bitcoin Prices Every Minute 12/24/2017')
plt.xlabel('Minute from Midnight')
plt.ylabel('Price (USD)')

Pretty noisy, right? Maybe a lot of last-minute Christmas gifts made the price even more unpredictable!

In this exercise, you will implement two length-51 moving average filters on this Bitcoin price data. The first will be causal and the second will be non-causal. Mathematically, we can represent these systems as follows:

$$
y_1[n] = \frac{1}{51}\sum_{i=0}^{50}x[n - i]
$$

$$
y_2[n] = \frac{1}{51}\sum_{i=-25}^{25}x[n - i]
$$

Furthermore, since the moving average filter is an LTI system we may implement it as a convolution using its impulse response. If you are having trouble seeing this, we suggest considering a length-5 moving average filter and "unrolling" the sum to understand that system definition and its impulse response.

Notice that the non-causal filter will require us to access negative indices according to the impulse response of our filter. A natural question to ask is how does the $\textrm{signal.convolve()}$ function perform non-causal convolution? How can you indicate negative indices when making an array for a system's impulse response? This is where the "same" mode comes in! We may use the "same" mode as follows:

```
y = signal.convolve(x, h, 'same'),
```

where $x$ is of length $N$ and $h$ is length $M$. This line of code will first perform regular convolution like the default mode where the first sample of each sequence is assumed to be at $n=0$. Then, it will only keep the center $N$ values (length of first argument/array). This operation is equivalent to zero-centering the second argument/array (```h``` in the above example). You may want to try a couple small examples to convince yourself this is true. The "same" mode will be important to keep in mind throughout this lab and the rest of the course.

**Important Note:** For the following two parts, we have provided the appropriate start and end indices to help us make sure each implementation returns results of the same size and to remove initial condition worries (ramping behavior since we would have fewer than 51 samples as the filter has partial overlap). **Please follow the plotting instructions in the following parts carefully!**

a. Construct the causal filter and apply it to the provided bitcoin price data (apply your causal system to the data in the ``bitcoin_data`` variable). To make sure your output is the same length and matches up correctly in time, please slice the result of convolving ``bitcoin_data`` with your causal filter using ``start`` and ``end``. Note that we do this to create the ``plotting_data`` variable. Plot the original data (``plotting_data``) and your smoothed data on the same plot. **Don't forget to include a legend!**

b. Construct the non-causal filter and apply it to the provided bitcoin price data (``bitcoin_data`` variable). Perform the same ``start`` and ``end`` slicing on your result as in part (a). Plot the original data (``plotting_data``) and your smoothed data on the same plot.

c. Plot the difference signals for each filter on the same plot. Let the difference signal for a system's output be given by

$$
y_d = y - \hat{y},
$$

where $\hat{y}$ is your system output and $y$ is the sliced original data used for plotting, ``plotting_data``.

d. Comment on the results in the following Markdown cell. What is noticeably different between the two sets of smoothed results from the causal and non-causal systems? Is it helpful to know a lot of past information or a decent amount of past and future information?

In [ ]:
# Provided code
L = 51
half_L = 25
start = 50
end = len(bitcoin_data)-half_L
plotting_data = bitcoin_data[start:end] # plot against this data in parts a/b, pay attention to how we slice
result_length = len(plotting_data)

# Code for 2.a here, don't forget to plot original and filtered signals on same plot!
ha = np.ones(51) / 51
y = signal.convolve(bitcoin_data, ha)
n = np.arange(start, end)
plt.figure(figsize=(6,8))
plt.plot(n, y[start:start+result_length], label = 'after filter (causal)')
plt.plot(n, plotting_data, label = 'before filter')
plt.title('Bitcoin prices: original vs causal filter')
plt.xlabel('Minute from midnight')
plt.ylabel('Price(USD)')
plt.legend()

# Code for 2.b here
y2 = signal.convolve(bitcoin_data, ha, 'same')
plt.figure(figsize=(6,8))
plt.plot(n, y2[start:end], label = 'after filter (non-causal)')
plt.plot(n, plotting_data, label = 'before filter')
plt.title('Bitcoin prices: original vs non-causal filter')
plt.xlabel('Minute from midnight')
plt.ylabel('Price(USD)')
plt.legend()

        
    
# Code for 2.c here
dy = plotting_data - y[start:start+result_length]
dy2 = plotting_data - y2[start:end]
plt.figure(figsize=(6,8))
plt.plot(n, dy, label = 'original - causal')
plt.plot(n, dy2, label = 'original - non-causal')
plt.title('Differences of original data vs filtered data')
plt.xlabel('Minute from midnight')
plt.ylabel('Difference (USD)')
plt.legend()



## Comments for here:

Part 2(d):
As seen in the differences part, the non-causal filter performs better than the causal filter, as clearly seen in the differences graph. The casual filter uses the past 50 samples, so the curve doesn't align directly with the original signal; it is shifted a little bit. On the other hand, the non-casual one averages 25 signals in the past and future, so its prediction is much more reliable, and the data is not shifted. 

# Image Convolution

In ECE 310, we typically focus on the implications of applying LTI systems in the frequency domain of a 1D signal. Our most common idea of a 1D signal is a piece of audio. In this section, we will experiment with image filtering along both axes of an image and see that we can do more than just filtering with convolution.

# Exercise 3: 1D Image Convolution

Apply a 1D length 11 moving average filter on the provided ``test-image.jpg`` image along the:

**Note: we will use the 'same' mode again when using $\textrm{signal.convolve()}$ for this exercise.**

a. Rows 

b. Columns

c. Rows then columns (**Hint**: make sure to convolve first along the rows, then use the result of this row-convolution to convolve along the columns!)

d. Columns then rows (**Hint**: make sure to convolve first along the columns, then use the result of this column-convolution to convolve along the rows!)

Plot each of the resulting images and give them unique titles.

e. Comment on the images from the "rows then columns" and "columns then rows" procedures. Are they the same? Explain your answer, why are they the same or different?

In [ ]:
# make filter and load image
image = imread('test-image.jpg')
plt.figure(figsize=(10,6))
plt.imshow(image, 'gray')
plt.title('Original Image')
L = 11
h = np.ones(L) * 1/11 # impulse response of length-11 moving average filter
n_rows, n_cols = image.shape
# Code for 3.a along rows (apply filter to each row independently)
image_row = np.zeros(image.shape)
# Hint: image_row[i, :] = signal.convolve(image[i, :], h, 'same') for each row
for i in range(image_row.shape[0]):
    image_row[i, :] = signal.convolve(image[i, :], h, 'same')
plt.figure(figsize=(10,6))
plt.imshow(image_row, 'gray')
plt.title('Convolve along x axis Image')
# along the columns (3.b)
image_col = np.zeros(image.shape)
for i in range(image_col.shape[1]):
    image_col[:, i] = signal.convolve(image[:, i], h, 'same')
plt.figure(figsize=(10,6))
plt.imshow(image_col, 'gray')
plt.title('Convolve along y axis Image')
# rows then columns (3.c)
image_rowcol = np.zeros(image.shape)
for i in range(image_rowcol.shape[0]):
    image_rowcol[i, :] = signal.convolve(image[i, :], h, 'same')
for i in range(image_rowcol.shape[1]):
    image_rowcol[:, i] = signal.convolve(image_rowcol[:, i], h, 'same')

plt.figure(figsize=(10,6))
plt.imshow(image_rowcol, 'gray')
plt.title('Convolve along x and then y axis Image')
# columns then rows (3.d)
image_colrow = np.zeros(image.shape)
for i in range(image_colrow.shape[1]):
    image_colrow[:, i] = signal.convolve(image[:, i], h, 'same')
for i in range(image_colrow.shape[0]):
    image_colrow[i, :] = signal.convolve(image_colrow[i, :], h, 'same')

plt.figure(figsize=(10,6))
plt.imshow(image_colrow, 'gray')
plt.title('Convolve along y and then x axis Image')

## Comments here

Part 3(e):
Both convolving along the x and then y axis and y and then x axis should be the same as seen in the pictures. They should be the same because convolution averages the 11 pixels along the x-axis first, and then averages the 11 pixels along the y-axis, which effectively averages the 11 by 11 area around each pixel. Therefore, flipping the sequence to the y-axis first and then the x-axis should have the same effect as averaging the 11*11 area. 


# Exercise 4: Building an Edge Detector

Let's now apply image convolution to perform edge detection. We will build a simple edge detector step-by-step using the following simple 1D filter:

$$
h[n] = \delta[n] - \delta[n-1]
$$

a. Intuitively or mathematically, what does this filter do to an input signal? In other words, what parts of a signal would give a strong (large magnitude) response and what parts would give a weak (small magnitude) response? You may answer this with a couple signal examples and the result of convolution with $h[n]$ or qualitative intuition.

b. Is this filter causal? Why or why not? In general, is it a problem if an image filter is non-causal? (Hint: consider the contexts in which we can or cannot violate causality.)

**Note**: For the next two parts, please store your results in separate variables. This will make part (e) much cleaner.

c. Apply $h[n]$ along the rows of the ``test-image.jpg`` image. Plot the result with a grayscale color mapping.

d. Apply $h[n]$ along the columns of the ``test-image.jpg`` image. Plot the result with a grascale color mapping.

So far we have checked for edge-like features in the image going along the rows and columns. Imagine these two results as indicating edge strength along the row axis (vertical edges) and column axis (horizontal edges) of the image, respectively. Take a minute to look at the differences between these two resulting images. Can you tell which one is detecting edges within a row and which one is doing so within a column? What would be a sensible way to incorporate these two dimensions of information? Imagine they form a 2D vector and take the norm! More precisely:

$$
I_F(r,c) = \sqrt{\left(I_R(r,c)\right)^2 + \left(I_C(r,c)\right)^2},
$$

where $I_R$ and $I_C$ are the row and column filtered results from parts (c) and (d) above, respectively.

e. Build the final result image $I_F$ according to the above equation. Plot the result again with a grayscale color mapping.

## Comments here
 
Part 4(a):   
This filter compares this signal and the last signal and calculates the differences between them. Effectively, as shown:   
y[n] = x[n]-x[n-1]  
This means that this signal would give a strong response when the change is big and a small response when the change is small. Acting like a differential in calculus.   
for example x[n] = [1 1 9]   
The result y[n] = [1,0,8], where 8 represents the huge jump from 1 to 9   

Part 4(b):   
This signal is causal because it only requires the current and past values. It is not a problem for an image filter to be non-causal because we are not actually dealing with time, we don't need to forsee future. However, in some special cases, such as real-time streaming, it might become an issue where we cannot process non casual filter.



In [ ]:
# load test-image.jpg
test = imread('test-image.jpg')
plt.figure(figsize=(10,6))
plt.imshow(test, 'gray')
plt.title('Original photo')
h = np.array([1, -1])
# Code for 4.c here:
test_row = np.zeros(test.shape)
for i in range(test.shape[0]):
    test_row[i, :] = signal.convolve(test[i, :], h, 'same')
plt.figure(figsize=(10,6))
plt.imshow(test_row, 'gray')
plt.title('Convolve along x axis Image')

# Code for 4.d here:
test_col = np.zeros(test.shape)
for i in range(test.shape[1]):
    test_col[:, i] = signal.convolve(test[:, i], h, 'same')
plt.figure(figsize=(10,6))
plt.imshow(test_col, 'gray')
plt.title('Convolve along y axis Image')


# Code for 4.e here:
test_final = np.sqrt(test_row**2 + test_col**2)
plt.figure(figsize=(10,6))
plt.imshow(test_final, 'gray')
plt.title('Image If')


## 2D Image Convolution

We don't need to limit ourselves to 1D image convolution. Our filters or "kernels" can be in two dimensions also! We will not spend much time on the math of 2D convolution/filtering in this class because it is best left for ECE 418 (Image and Video Processing); still,  we can use Python to try it out. But let's try something other than filtering this time!

Image convolution is not just for filtering or modifying an image. We can also use convolution to extract information from an image. Remember that convolution is is the process of "flipping and shifting" one signal over another signal. At each shift location, we perform a dot product (or inner product) to see how $\textit{similar}$ the signals are. A larger magnitude value at the output means the two signals are more similar. The following image illustrates 2D convolution.

<img src="convolution.jpg">

More formally, say we have a $3x3$ convolution kernel $\mathcal{K}$ where the center pixel is at index $(0,0)$, the result of the 2D convolution at pixel $(i,j)$ for image $I$, $O(i,j)$ is given by:

$$
O(i,j) = \sum_{k=-1}^{1}\sum_{l=-1}^{1}\mathcal{K}(k,l)I(i-k,j-l)
$$

Now, why is this useful? Suppose you want to design a system to recognize handwritten digits. How can you tell the difference between a "1" and a "4", for example? Think about how you as a human can separate these numbers! They both typically have one large vertical line down the middle, but we know we can differentiate them because a "4" has another shorter vertical line (depending how you draw it) and a horizontal line connecting them. This is where 2D convolution can help us! How about we create convolution kernels to highlight features we know to be discriminative, like horizontal and vertical lines.

The below code cell includes a function to perform 2D image convolution on a target image given a convolution kernel. We have also provided two 2D kernels: one for horizontal features and another for vertical features.

In [ ]:
def convolve_2d(image, kernel):
    result = signal.convolve2d(image, kernel, 'same')
    result[result < 0] = 0 # Keep values non-negative
    return result

# identify horizontal lines
horiz_kernel = np.array([[-2,-2,-2,-2,-2],
                         [1,1,1,1,1],
                         [1,1,1,1,1],
                         [1,1,1,1,1],
                         [-2,-2,-2,-2,-2]])

# identify vertical lines
vert_kernel = np.array([[-2,1,1,1,-2],
                        [-2,1,1,1,-2],
                        [-2,1,1,1,-2],
                        [-2,1,1,1,-2],
                        [-2,1,1,1,-2]])

In the folder for this lab, we have provided example images of the numbers "1", "4", and "8" from the popular MNIST dataset. These images are 28x28 and grayscale. Let's see what our filters can identify in the ``one.jpg`` image! Note the different scales on the feature image colorbars.

In [ ]:
one = imread('one.jpg')

plt.figure(figsize=(16,10))
plt.subplot(131)
plt.title('Original')
plt.imshow(one, 'gray')

one_horiz = convolve_2d(one, horiz_kernel)
plt.subplot(132)
plt.title('Horizontal Features')
plt.imshow(one_horiz, 'hot')
plt.colorbar(fraction=0.05)

one_vert = convolve_2d(one, vert_kernel)
plt.subplot(133)
plt.title('Vertical Features')
plt.imshow(one_vert, 'hot')
plt.colorbar(fraction=0.05)

# Exercise 5: 2D Image Convolution for Feature Detection

a. Create similar plots as the above example for the "1" image for the "4" (``four.jpg``) and "8" (``eight.jpg``) images in the following code cell.

b. Comment on the results and compare what is highlighted for each number.

c. What is the significance of having negative kernel values around the positive "feature highlighting" values? Think about what would happen if the negative values were zeros instead. Try playing around with the kernels or creating your own kernel if you are unsure.


In [ ]:
# Code for 5.a here
one = imread('four.jpg')

plt.figure(figsize=(16,10))
plt.subplot(131)
plt.title('Original')
plt.imshow(one, 'gray')

one_horiz = convolve_2d(one, horiz_kernel)
plt.subplot(132)
plt.title('Horizontal Features')
plt.imshow(one_horiz, 'hot')
plt.colorbar(fraction=0.05)

one_vert = convolve_2d(one, vert_kernel)
plt.subplot(133)
plt.title('Vertical Features')
plt.imshow(one_vert, 'hot')
plt.colorbar(fraction=0.05)

one = imread('eight.jpg')

plt.figure(figsize=(16,10))
plt.subplot(131)
plt.title('Original')
plt.imshow(one, 'gray')

one_horiz = convolve_2d(one, horiz_kernel)
plt.subplot(132)
plt.title('Horizontal Features')
plt.imshow(one_horiz, 'hot')
plt.colorbar(fraction=0.05)

one_vert = convolve_2d(one, vert_kernel)
plt.subplot(133)
plt.title('Vertical Features')
plt.imshow(one_vert, 'hot')
plt.colorbar(fraction=0.05)

## Comments here:

Part 5(b):  
In picture 4, the vertical features output shows the vertical lines in the number, and the  horizontal features outputs where the number has horizontal strokes.  
In picture 8, similar things happen; the vertical parts of the curve are highlighted in the vertical features graph, and the horizontal parts are highlighted in the horizontal graph.  

  
Part 5(c):  
The negative weight around the edges suppresses the response when the whole area is bright. This makes sure that the horizontal and vertical features can be separated. The horizontal kernel, on the top and bottom, makes sure that vertical strokes won't trigger the response, similar can be said for the vertical kernel. Without the negative values, both kernels will only output positive values, making it hard to distinguish horizontal and vertical features, and it also increases false positives since the edges might not be strong enough to suppress the response. 


For the final activity, we will explore an example of a non-linear system. First, a bit of background.

There are many different types of noise that can appear in images. One such type is salt-and-pepper noise. This noise occurs when pixels in a camera or an existing image become fully active or inactive. In other words, a normal pixel either takes on its minimum or maximum possible value. The following code cell shows an original image and a version of it that has been corrupted by 20% salt-and-pepper noise (20% of the pixels are affected). In this activity, we will see whether we can use our LTI systems from before to denoise our image.

In [ ]:
clean_image = imread('clean-image.jpg')
noisy_image = imread('noisy-image.jpg')
plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(clean_image, 'gray')
plt.title('Original Image')
plt.subplot(122)
plt.imshow(noisy_image, 'gray')
plt.title('Image with 20% Salt-and-Pepper Noise')

# Exercise 6: Non-Linear Systems are Cool Too!

We will attempt to use two different filters: a 5x5 mean filter and a 5x5 median filter. Note that a median filter is a non-linear system! A 5x5 median filter simply takes the median of the 25 pixels surrounding the center pixel in the filter and assigns that value to the center pixel.

a. Explain/prove why the median filter is a non-linear system. If you don't know where to start, try showing a counter-example for how a one-dimensional median filter fails the test for linearity. 

b. Apply a 5x5 mean filter to the noisy image and plot the result. You can do this two different ways. You can apply a length-5 mean filter along the rows and columns in any order or use our $\textrm{convolve_2d()}$ function from before with an appropriate filter you create.

c. Apply a 5x5 median filter to the noisy image and plot the result. Use $\textrm{signal.medfilt()}$ to perform the filtering. Look up the scipy documentation for notes on this function's usage.

d. Comment on the differences. Which filter seems to work better? Why do you think so?

In [ ]:
# Code for 6.b
mean_ker = np.ones((5,5))/25
mean_image = convolve_2d(noisy_image, mean_ker)
plt.figure(figsize=(16,10))
plt.title('5x5 Mean Filtered Image')
plt.imshow(mean_image, 'gray')

# Code for 6.c
med_image = signal.medfilt(noisy_image, kernel_size = (5,5))
plt.figure(figsize=(16,10))
plt.title('5x5 Median Filtered Image')
plt.imshow(med_image, 'gray')



## Comments here

Part 6(a):
The median filter is a nonlinear system that can be seen in the example below:  
x1[n] = [1,2,3] y1[n] = [2]  
x2[n] = [1,-10,1] y2[n] = [1]  
x1[n] + x2[n] = [2,-8,4] which is not equal to y1[n] + y2[n] = [3], the correct ansxwer should be 2  
This proves that this system is non-linear because a linear system should have 2 properties. 1. When you add inputs, the output should be the 2 original outputs added together, and 2. When you multiply input by a constant, the output should be multiplied by the same constant as well. The above example doesn't follow rule 1. 

Part 6(d):  
The median filter seems to work much better because there are way fewer distortion dots in the picture. I think this is the case because the median filter actually chooses one of the pixels in the original pictures, while the mean filter can choose values that are not in the original pictures. Furthermore, median filters are less affected by outliers, since salt and pepper distortion is full of extreme values, the mean filter can really mess up as well. 

## Submission Instructions

Make sure to place all image and data files along with your .ipynb lab report (this file) in one folder, zip the folder, and submit it to Canvas under the Lab 2 assignment. Please name the zip file ``<netid>_Lab2``.